In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

import torchvision.datasets as datasets

LSVRC_train = datasets.ImageFolder(root='../data/ImageNet-1K/train', transform=transform)

In [ ]:
import mobileclip
model_clip, _, preprocess = mobileclip.create_model_and_transforms('mobileclip_s0', pretrained='../models/mobileclip_s0.pt')
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')
device = 'cuda:0'
model_clip = model_clip.eval().to(device)

In [ ]:
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
trainset_loader = DataLoader(LSVRC_train, batch_size=128, shuffle=False, num_workers=8)

def get_features(model_clip, dataloader):
    all_features = []

    with torch.no_grad():
        for images, _ in tqdm(dataloader):
            features = model_clip.encode_image(images.to(device))

            all_features.append(features)

    return torch.cat(all_features).cpu().numpy()

LSVRC_features = get_features(model_clip, trainset_loader)

In [ ]:
LSVRC_features.shape

In [ ]:
torch.save(LSVRC_features, '../data/trainset_128.pt')